### Modelagem e Refinamento de Regra de Negócio
Esse projeto será construido levando em consideração o modelo de arquitetura sugerido na seguinte documentação ->
[Medallion Schema](https://www.databricks.com/sites/default/files/inline-images/building-data-pipelines-with-delta-lake-120823.png?v=1702318922)

Nessa camada executamos:
- Construção de entidades relacionadas
- Agregações e Construção de Indicadores
- Disponibilização de tabelas refinadas para o BI (Área de Negócio)

In [0]:
%run ../../Lakehouse/functions/functions

#### Variáveis de Path - Camada Tratada Prata

In [0]:

path_silver_diesel_gnv = '/databricks/silver/anp/tb_diesel_gnv/'
path_silver_etanol_gasol = '/databricks/silver/anp/tb_etanol_gasolina/'
path_silver_glp = '/databricks/silver/anp/tb_glp/'

#### Case de BI 
O cliente gostaria de ter algumas informações já agregadas/consolidadas acerca de preços praticados nesse semestre e te passou uma lista dos indicadores que ele precisa avaliar para chegar a esse resultado:

- **Preço Médio, Minimo e Máximo práticados na Região**
- **Preço Médio, Minimo e Máximo por Bandeira**
- **Total de Clientes (CPNJs) por Bairro/Municipio e seus tickets médios**

In [0]:
# Leitura dos dados a partir de seus paths
df_diesel_gnv = spark.read.format('delta').load(path_silver_diesel_gnv)
df_etanol_gasol = spark.read.format('delta').load(path_silver_etanol_gasol)
df_glp = spark.read.format('delta').load(path_silver_glp)

# Unificação dos dados carregados
df_geral = df_diesel_gnv.unionByName(df_etanol_gasol)\
                        .unionByName(df_glp)


### Agregação dos Indicadores

- Região 
- Unidade de Médida do Combustível
- Produto

In [0]:
df_matriz_precos_praticados = df_geral.groupby(
        'Produto',
        'Unidade_de_Medida',
        'Regiao_Nome'
    )\
    .agg(
        round(avg(col('Valor_de_Venda')),2).alias('Preco_Medio'),
        min(col('Valor_de_Venda')).alias('Menor_Preco'),
        max(col('Valor_de_Venda')).alias('Maior_Preco')
    )

#### Gravação da Tabela Preços Regionais

In [0]:
df_matriz_precos_praticados\
    .write.format('delta')\
    .mode('overwrite')\
    .saveAsTable('default.tb_precos_regiao_produto')


### Preços por Bandeira de Posto

In [0]:
df_matriz_precos_bandeira = df_geral.groupby(
        'Produto',
        'Bandeira'
    )\
    .agg(
        round(avg(col('Valor_de_Venda')),2).alias('Preco_Medio'),
        min(col('Valor_de_Venda')).alias('Menor_Preco'),
        max(col('Valor_de_Venda')).alias('Maior_Preco')
    )

#### Gravação da Tabela Preços Regionais

In [0]:
df_matriz_precos_bandeira\
    .write.format('delta')\
    .mode('overwrite')\
    .saveAsTable('default.tb_precos_bandeiras')

### Quantidade de Clientes por Bairro e Municipio

In [0]:
df_clientes = df_geral.groupby(
        'Municipio',
        'Bairro'
    )\
    .agg(
        round(avg(col('Valor_de_Venda')),2).alias('Ticket_Medio'),
        count_distinct(col('CNPJ_da_Revenda')).alias('Qtd_Clientes_CNPJ')

    )\
        .withColumn('Tipo_Cliente', when(col('Qtd_Clientes_CNPJ')>=15,lit('VIP'))\
                                  .when((col('Qtd_Clientes_CNPJ')>=8) & (col('Qtd_Clientes_CNPJ')<15),lit('Premium'))\
                                    .otherwise(lit('Basic'))
                                    )

#### Gravação da Tabela de Clientes Relevantes

In [0]:
df_clientes\
    .write.format('delta')\
    .mode('overwrite')\
    .saveAsTable('default.tb_clientes_relevantes')